In [284]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

%load_ext autoreload
%autoreload 2

from utils.utils import (
    generate_data,
    print_project_summary,
    student_assignment
)

file_location = '../data/Winter 25 Matching.xlsx'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [285]:
application_df = pd.read_excel(file_location, sheet_name="Application Before Editing")

application_df.head()


,Timestamp,Email Address,Full Name,ChicagoID from the back of your ID card (8 numbers + letter). This is NOT the same as your student ID number.,GitHub Username,Are you currently enrolled in the Data Science Clinic or have you taken the clinic in a previous quarter?,"If you are currently enrolled or have taken the clinic in a previous quarter, on which project did you work?",Current Degree Program,Academic Program / Concentration,Please upload a current resume:,...,Please rank your familiarity with GitHub:,Please rank your competency in Python:,"Some partners require short (30-60 min), non-technical interviews. Are you willing to interview via Zoom?",Many of the projects require having access to a computer with administrator privileges. Do you have access to a computer?,"Outside of class, do you have a job, internship or other requirement during the quarter which would limit your ability to meet with students and mentors between 9am-5pm in Hyde Park? If so, what?",Project 1,Project 2,Project 3,Project 4,Project 5
0,2024-11-21 14:38:08.154,johminjae@uchicago.edu,Minjae Joh,71196121E,mjayjoh,No,NaN,Undergrad: 3rd year,Data Science,https://drive.google.com/open?id=15iGUhwSsFRdE...,...,4.0,4.0,Yes,Yes,No,University of Rwanda,Kids First Chicago,UChicago Library,Morningstar*,Argonne*
1,2024-11-21 14:45:23.875,malls@uchicago.edu,Soham Mall,90680615P,soham627,No,NaN,MA or MS 1st year,Computer Science,https://drive.google.com/open?id=1rSbHhDcJBwr1...,...,4.0,5.0,Yes,Yes,No,Fermi - Simulations*,Invenergy*,Argonne*,Fermi - GNN*,Morningstar*
2,2024-11-21 16:25:29.691,victorqian@uchicago.edu,Haohui Qian,30902327S,VictorQian03,No,NaN,Undergrad: 4th year,Statistics,https://drive.google.com/open?id=1MIS6fBlEV6Su...,...,4.0,5.0,Yes,Yes,None at the moment,Morningstar*,Fermi - Simulations*,Fermi - GNN*,Argonne*,Invenergy*
3,2024-11-21 17:20:15.030,estherli208@uchicago.edu,Zhiduan Li,12410814,Eswelt,No,NaN,MA or MS 1st year,MA Public Policy (MPP),https://drive.google.com/open?id=1raxcrtYNc-PQ...,...,1.0,4.0,Yes,Yes,No,Building Decarbonization Coalition*,IDI - Grievances,International Rescue Committee,UChicago Library,Internet Equity Initiative
4,2024-11-21 17:41:54.223,jspinoglio@uchicago.edu,Jennifer Spinoglio,21875522L,jenspinoglio,Yes,RAFI - Grocery,Undergrad: 4th year,Data Science,https://drive.google.com/open?id=1_Sc02qY1rI5g...,...,4.0,5.0,No,Yes,NaN,RAFI - Grocery,RAFI - Grocery,Food System 6,Kids First Chicago,UChicago Library


In [286]:
prev_proj_col = "If you are currently enrolled or have taken the clinic in a previous quarter, on which project did you work?"
def strip_chars(df):
    cols = ['Project 1', 'Project 2', 'Project 3', 'Project 4', 'Project 5', prev_proj_col]
    for col in cols:
        df[col] = df[col].str.replace('*', '')

    return df

application_df = strip_chars(application_df)

In [287]:
def generate_priorities(df):
    # Default everyone to low priority
    df["Priority"] = "low"

    def adj_priority(filter, priority):
        df.loc[filter, 'Priority'] = priority

    # Adjust priority for third year data science majors
    adj_priority(
        (df["Current Degree Program"] == "Undergrad: 3rd year") &
        (df['Academic Program / Concentration'].str.contains(r'data science', na=False, case=False)),
        "med"
    )

    # Adjust priority for general fourth years (non-DS majors)
    adj_priority(
        (df["Current Degree Program"] == "Undergrad: 4th year"),
        "med"
    )

    # Adjust priority for second year masters students
    adj_priority(
        (df["Current Degree Program"] == "MA or MS 2nd year"),
        "med-high"
    )

    # Adjust priority for fourth year data science majors
    adj_priority(
        (df["Current Degree Program"] == "Undergrad: 4th year") &
        (df['Academic Program / Concentration'].str.contains(r'data science', na=False, case=False)),
        "high"
    )

    # Adjust priority for fifth year data science majors
    adj_priority(
        (df["Current Degree Program"] == "Undergrad: 5th year") &
        (df['Academic Program / Concentration'].str.contains(r'data science', na=False, case=False)),
        "high"
    )

    return df

application_df = generate_priorities(application_df)


In [288]:
cscol1 = 'If you have taken an introduction to computer science / "Computer Science 1" course (such as CMSC 141, 151 or 161), please list that course here. DATA courses do not count.'
cscol2 = 'If you have taken a course that would be considered the equivalent of "Computer Science 2" (such as CMSC 142, 152 or 162), please list that course here. DATA courses do not count.'

def generate_cs_column(df):
    df["Strong CS"] = "No"

    def adjust_col(filter, column, newvalue):
        df.loc[filter, column] = newvalue

    # Adjust CS strength
    adjust_col(
        (df[cscol1].notna() & df[cscol2].notna()),
        'Strong CS',
        "Yes"
    )
    return df


application_df = generate_cs_column(application_df)

application_df["Strong CS"].value_counts()

Strong CS
No     70
Yes    40
Name: count, dtype: int64

In [289]:
prev_col = "Are you currently enrolled in the Data Science Clinic or have you taken the clinic in a previous quarter?"
prev_proj_col = "If you are currently enrolled or have taken the clinic in a previous quarter, on which project did you work?"

returning_students = application_df[application_df[prev_col] == "Yes"][['Email Address', prev_proj_col]].values

# Use this to force specific students onto specific projects.
forced_assignments = {el[0]: el[1] for el in returning_students}

In [290]:
deprioritized_students = ["hanyunl@uchicago.edu"]

for student in deprioritized_students:
    if student in forced_assignments:
        del forced_assignments[student]

In [291]:
forced_assignments



{'jspinoglio@uchicago.edu': 'RAFI - Grocery',
 'alexan@uchicago.edu': 'Internet Equity Initiative',
 'msaidenberg@uchicago.edu': 'Fermi - GNN',
 'emmaricks@uchicago.edu': 'International Rescue Committee',
 'mlumlauf@uchicago.edu': 'International Rescue Committee',
 'klin665@uchicago.edu': 'Fermi - Simulations',
 'siyakalra@uchicago.edu': 'Argonne',
 'lilaweizer@uchicago.edu': 'Food System 6',
 'alexbacro@uchicago.edu': 'Fermi - GNN',
 'kaitlynli2025@uchicago.edu': 'UChicago Library',
 'shaileshbolduc@uchicago.edu': 'UChicago Library',
 'bellegn@uchicago.edu': 'International Rescue Committee',
 'nitikakurma@uchicago.edu': 'UChicago Library',
 'kevinl25@uchicago.edu': 'Morningstar',
 'ygu0929@uchicago.edu': 'Fermi - GNN',
 'amizuno@uchicago.edu': 'Morningstar',
 'annapoon@uchicago.edu': 'CMAP',
 'isaacharlem@uchicago.edu': 'Argonne',
 'jchefetz@uchicago.edu': 'Internet Equity Initiative',
 'khwaishvohra@uchicago.edu': 'RAFI - Grocery',
 'jbendavid@uchicago.edu': 'Center for Living System

In [292]:
priority_mapping = {'high': 1, 'med-high': 2, 'med': 3, 'low': 4}

max_students = {'Fermi - GNN': 3}


# List of technical projects
technical_project_list = [
    'Morningstar', 'Fermi - GNN', 'Fermi - Simulations', 'Center for Living Systems',
    'Building Decarbonization Coalition', 'Argonne', 'CMAP', 'Invenergy'
]

priority_weights = [10, 15, 25, 30]

data_to_process_dict = generate_data(application_df, technical_project_list, priority_mapping)

Dropping 0 students because they do not have a computer and not required
Total Students available for matching: 110


In [ ]:
all_project_list = data_to_process_dict.pop('all_project_list')
assignment_df = student_assignment(priority_weights = priority_weights, **data_to_process_dict,
                                   max_students_dict=max_students, preassigned_students=forced_assignments,
                                   number_of_projects_to_run=None, drop_projects=["University of Northern Iowa"], verbose=False)
print(f"\nAverage Ranking: {assignment_df.Ranking.mean():.2f}")
print(assignment_df.Ranking.value_counts())
print_project_summary(assignment_df, all_project_list)

assignment_df.to_csv('../data/test_1.csv', index=False)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/hannifan/projects/the-clinic/.venv/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/sr/lrm99gp93rx47xspcx2z49gr0000gp/T/6948200b282240c1a3d5dde0933e600d-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/sr/lrm99gp93rx47xspcx2z49gr0000gp/T/6948200b282240c1a3d5dde0933e600d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2056 COLUMNS
At line 17621 RHS
At line 19673 BOUNDS
At line 21450 ENDATA
Problem MODEL has 2051 rows, 1776 columns and 10252 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.02


Average Ranking: 3.02
Ranking
1.0      48
3.0       7
2.0       5
4.0       3
100.0     1
5.0       1
Name: count, 

,Project Name,Number,High Priority,High-Med Priority,Med Priority,Low Priority,Experienced,Rankings,Average Ranking
0,International Rescue Committee,6,6,0,0,0,2,"1,1,1,1,1,2",1.17
1,Argonne,5,5,0,0,0,2,"1,1,1,1,1",1.00
2,Fermi - GNN,5,5,0,0,0,2,"1,1,1,2,3",1.60
3,Center for Living Systems,5,5,0,0,0,1,"1,1,1,3,4",2.00
4,UChicago Library,5,4,0,1,0,2,"1,1,1,3,5",2.20
5,CMAP,5,4,1,0,0,2,"1,1,2,4,100",21.60
6,Food System 6,4,4,0,0,0,1,"1,1,1,1",1.00
7,Internet Equity Initiative,4,4,0,0,0,0,"1,1,1,2",1.25
8,Fermi - Simulations,4,3,1,0,0,2,"1,1,1,3",1.50
9,IDI - Grievances,4,3,1,0,0,2,"1,1,1,3",1.50


In [294]:
project_to_comp = 'Kids First Chicago'

list_of_not_assigned = list(assignment_df.loc[ (assignment_df.Priority == 1) & (assignment_df.Ranking.isna()), 'Email Address'])
prefs_df = data_to_process_dict['ranking'].copy()
prefs_df.loc[~(prefs_df.Ranking == 100.0), :].copy()

unmatched_prefs = prefs_df.loc[(prefs_df.loc[:, 'Project Name'] == project_to_comp) & prefs_df.loc[:, 'Email Address'].isin(list_of_not_assigned)
             & (prefs_df.Ranking < 100), :]

unmatched_info = pd.merge(unmatched_prefs, assignment_df.loc[:, ['Email Address', 'Experienced', 'Priority']], how='left', on='Email Address')

matched = assignment_df.loc[(assignment_df.loc[:, 'Project Assigned'] == project_to_comp), :]
display(matched.sort_values('Ranking'))

display(unmatched_info)


,Email Address,Priority,Experienced,Project Assigned,Ranking


,Email Address,Project Name,Ranking,Experienced,Priority


In [295]:
data_to_process_dict['student_characteristics'].Priority.value_counts()


Priority
1    65
2    23
4    14
3     8
Name: count, dtype: int64

In [296]:
assignment_df[(assignment_df['Priority'] == 1) & (assignment_df['Project Assigned'].isna())]

,Email Address,Priority,Experienced,Project Assigned,Ranking
9,andresg@uchicago.edu,1,False,None,None
21,dariushinojosa@uchicago.edu,1,False,None,None
23,ddanaie@uchicago.edu,1,False,None,None
47,jeraldshu@uchicago.edu,1,False,None,None
49,jmdeleon@uchicago.edu,1,False,None,None
69,marchakitus@uchicago.edu,1,False,None,None
